# Patient Summary Generator

This notebook generates a comprehensive patient summary from EMBR format data (optimized FHIR data) using OpenAI's GPT model.

## Process
1. Loads environment variables including OpenAI API key
2. Reads patient data from .embr files
3. Processes the data using a customized prompt
4. Generates a structured summary including:
   - Demographics
   - Conditions
   - Medications
   - Care plans
   - Procedures
   - Key encounters and observations

The summary maintains strict adherence to the provided data without inference or external recommendations.

In [25]:
import openai
from pathlib import Path
from dotenv import load_dotenv
import os
from IPython.display import display, Markdown

# Load environment variables
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [26]:
# Get first .embr file
embr_files = list(Path("../patients").glob("*.embr"))

# Arbitrary patient file, Adjust from 0-99 for different patients
with open(embr_files[11], 'r') as f:
    patient_data = f.read()

In [27]:

# Edit this prompt to fit your needs
prompt = f"""
You are analyzing patient data in EMBR format, an optimized summary of FHIR data. The data includes essential information.

Provide a summary with the following information:
Patient demographics
Conditions
Medications and MedicationRequests
Care plan and goals
Key procedures
Important Encounters and Observations


Only use the data provided, without adding, recommending, or inferring beyond it.
Reference any data with the dates and ID (e.g. EN-129 for Encounter 129).

Patient Data:
{patient_data}
"""

##### In this cell, we make an API call to OpenAI's GPT-4 model to analyze and summarize patient data. The model is given a system prompt identifying it as a healthcare professional, and a user prompt containing:
- The patient data in EMBR format
- Instructions to summarize specific aspects like demographics, conditions, medications etc.
- Guidelines to only use provided data and reference data points with dates/IDs

The response is stored in the 'summary' variable which contains the generated patient summary text.


In [32]:
response = openai.chat.completions.create(
    model="gpt-4o-mini", # Not all files can be summarized with gpt-4o, as the tokens are too high
    messages=[
        {"role": "system", "content": "You are a healthcare professional skilled at analyzing and summarizing patient records."},
        {"role": "user", "content": prompt}
    ],
    temperature=0.7,
    max_tokens=5000
)

summary = response.choices[0].message.content
display(Markdown(summary))


### Patient Summary

**Patient Demographics:**
- **Name:** Mr. Chas Clark Green
- **Gender:** Male
- **Date of Birth:** October 30, 1985
- **Address:** 244 Mohr Club Unit 36, Hudson, MA, 01749
- **Marital Status:** Never Married
- **Communication Language:** English (United States)

---

**Conditions:**
- **Confirmed Conditions:**
  - Risk activity involvement (EN-2, 2001-12-12)
  - Medication review due (EN-8, 2003-12-24)
  - Prediabetes (EN-15, 2008-01-02)
  - Stress (EN-15, 2008-01-02, resolved 2017-01-11)
  - Anemia (EN-26, 2011-01-05)
  - Acute viral pharyngitis (EN-42, 2016-01-08)
  - Viral sinusitis (EN-49, 2016-10-19)
  - Obesity (BMI 30+) (EN-58, 2016-10-26)
  - Gingivitis (EN-58, 2016-10-26, resolved 2016-11-16)
  - Sleep disorder (EN-101, 2016-11-02)
  - Sleep apnea (EN-139, 2016-12-05)
  - Essential hypertension (EN-155, 2017-01-11)
  - Metabolic syndrome X (EN-549, 2022-02-09)
  - Injury of knee (EN-666, 2023-06-19)
  - Rupture of patellar tendon (EN-680, 2023-06-26)
  - Injury of medial collateral ligament of knee (EN-778, 2024-07-10)

---

**Medications and Medication Requests:**
- **Active Medication:**
  - Sodium fluoride 0.0272 MG/MG Oral Gel (MD-122, active)
- **Medication Requests (Stopped):**
  - Amoxicillin 250 MG / Clavulanate 125 MG Oral Tablet (MR-51, 2016-10-19)
  - Hydrochlorothiazide 25 MG Oral Tablet (multiple requests, last on MR-765, active as of 2024-02-21)
  - Ibuprofen 200 MG Oral Tablet (MR-459, 2020-05-19)
  - Naproxen sodium 220 MG Oral Tablet (MR-670, 2023-06-19; MR-782, 2024-07-10)
  - Meperidine Hydrochloride 50 MG Oral Tablet (MR-683, 2023-06-26)
  - Acetaminophen 325 MG / HYDROcodone Bitartrate 7.5 MG Oral Tablet (MR-795, 2024-07-18)

---

**Care Plans and Goals:**
- **Diabetes Self-Management Plan:** 
  - Active since January 2, 2008 (CP-21).
- **Weight Reduction Regimen and CPAP Treatment for Sleep Apnea:**
  - Active since December 5, 2016 (CP-150).
- **Lifestyle Education Regarding Hypertension:** 
  - Active since January 11, 2017 (CP-208).
- **Wound Care for Laceration of Foot:**
  - Completed on May 19, 2020 (CP-463).
- **Physiotherapy Care Plan for Ruptured Patellar Tendon:**
  - Completed on June 19, 2023 (CP-674).
- **Physiotherapy Care Plan for Injury of Medial Collateral Ligament:**
  - Completed on July 10, 2024 (CP-786).

---

**Key Procedures:**
- **Sleep Apnea Assessments:**
  - Multiple assessments performed between November 2016 and February 2024.
- **Dental Procedures:**
  - Multiple dental consultations and care performed, including plaque removal and fluoride treatments from November 2016 to March 2023.
- **Emergency Room Admissions:**
  - Laceration treatment on May 19, 2020 (PR-458).
  - Knee injuries treated on June 19, 2023 (PR-669) and July 10, 2024 (PR-781).
- **Hospital Admissions for Joint Surgery:**
  - Non-urgent orthopedic admission for knee rupture on June 26, 2023 (PR-681).
  - Postoperative care for knee surgery on June 29, 2023 (PR-682).

---

**Important Encounters and Observations:**
- **Encounters:**
  - Regular general examinations and follow-ups with Dr. Rosa Caraballo and Dr. Araceli Willms from 2001 to 2024.
  
- **Observations:**
  - Vital signs and lab results consistently monitored, including weight, blood pressure, glucose levels, and hemoglobin A1c measurements at various encounters.
  - Notable blood pressure readings indicate hypertension with systolic values up to 151 mm(Hg) and diastolic values reaching 118 mm(Hg).

--- 

This summary reflects the patient's ongoing healthcare management, highlighting key health conditions, treatment plans, encounters, and significant laboratory observations.